<a href="https://colab.research.google.com/github/yrodriguezmd/IceVision_in_the_works/blob/main/IV_plant%2C_customparser_2021_8_28_1pm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/airctic/icevision/master/install_colab.sh
!chmod +x install_colab.sh && ./install_colab.sh

--2021-08-28 19:27:58--  https://raw.githubusercontent.com/airctic/icevision/master/install_colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1244 (1.2K) [text/plain]
Saving to: ‘install_colab.sh’

install_colab.sh    100%[===================>]   1.21K  --.-KB/s    in 0s      

2021-08-28 19:27:59 (67.4 MB/s) - ‘install_colab.sh’ saved [1244/1244]

Installing icevision + dependencices for CUDA 10
Uninstalling some dependencies to prevent errors
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102
Found existing installation: fastai 1.0.61
Uninstalling fastai-1.0.61:
  Successfully uninstalled fastai-1.0.61
Installing some dependencies to prevent errors
     |██

In [1]:
from icevision.all import *
import pandas as pd

INFO     - Downloading default `.ttf` font file - SpaceGrotesk-Medium.ttf from https://raw.githubusercontent.com/airctic/storage/master/SpaceGrotesk-Medium.ttf to /root/.icevision/fonts/SpaceGrotesk-Medium.ttf | icevision.visualize.utils:get_default_font:69
INFO     - Downloading mmdet configs | icevision.models.mmdet.download_configs:download_mmdet_configs:31


0B [00:00, ?B/s]

In [2]:
import icevision
print(icevision.__version__)
print(torch.__version__)
print(torchvision.__version__)

0.8.1
1.8.0+cu101
0.9.0+cu101


In [3]:
 dataset_name = 'plantdoc'

# go to github, code button, clone using https
!git clone https://github.com/pratikkayal/PlantDoc-Object-Detection-Dataset.git

Cloning into 'PlantDoc-Object-Detection-Dataset'...
remote: Enumerating objects: 5190, done.
remote: Total 5190 (delta 0), reused 0 (delta 0), pack-reused 5190
Receiving objects: 100% (5190/5190), 941.89 MiB | 44.63 MiB/s, done.
Resolving deltas: 100% (2328/2328), done.
Checking out files: 100% (5199/5199), done.


In [4]:
%cd PlantDoc-Object-Detection-Dataset/


/content/PlantDoc-Object-Detection-Dataset


In [5]:
!ls

LICENSE.txt  TEST  test_labels.csv  TRAIN  train_labels.csv


In [6]:
train_labels = pd.read_csv('train_labels.csv')
train_labels.rename(columns={'class':'label'}, inplace=True)
train_labels.head()

,filename,width,height,label,xmin,ymin,xmax,ymax
0,cherry-tree-leaves-and-fruits.jpg,350,300,Cherry leaf,198,77,299,252
1,cherry-tree-leaves-and-fruits.jpg,350,300,Cherry leaf,3,114,148,235
2,cherry-tree-leaves-and-fruits.jpg,350,300,Cherry leaf,30,184,189,297
3,cherry-tree-leaves-and-fruits.jpg,350,300,Cherry leaf,226,4,346,83
4,peach-and-leaf-stock-image-2809275.jpg,1300,1099,Peach leaf,237,479,527,810


In [7]:
train_labels.info()
# 8,469 images, no missing val

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  8469 non-null   object
 1   width     8469 non-null   int64 
 2   height    8469 non-null   int64 
 3   label     8469 non-null   object
 4   xmin      8469 non-null   int64 
 5   ymin      8469 non-null   int64 
 6   xmax      8469 non-null   int64 
 7   ymax      8469 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 529.4+ KB


In [10]:
print(train_labels.filename.nunique(), 'filenames')
# paper: 2,598 data points)
print(train_labels.label.nunique(), 'labels')
# paper: 13 plant species, 17 classes of diseases)

2345 filenames
29 labels


In [11]:
_CLASSES = train_labels.label.unique().tolist()
_CLASSES

['Cherry leaf',
 'Peach leaf',
 'Corn leaf blight',
 'Apple rust leaf',
 'Potato leaf late blight',
 'Strawberry leaf',
 'Corn rust leaf',
 'Tomato leaf late blight',
 'Tomato mold leaf',
 'Potato leaf early blight',
 'Apple leaf',
 'Tomato leaf yellow virus',
 'Blueberry leaf',
 'Tomato leaf mosaic virus',
 'Raspberry leaf',
 'Tomato leaf bacterial spot',
 'Squash Powdery mildew leaf',
 'grape leaf',
 'Corn Gray leaf spot',
 'Tomato Early blight leaf',
 'Apple Scab Leaf',
 'Tomato Septoria leaf spot',
 'Tomato leaf',
 'Soyabean leaf',
 'Bell_pepper leaf spot',
 'Bell_pepper leaf',
 'grape leaf black rot',
 'Potato leaf',
 'Tomato two spotted spider mites leaf']

In [12]:
class_map = ClassMap(_CLASSES)

In [13]:
class_map.get_by_name('Apple leaf')
# code to be used in custom parser

11

In [14]:
!ls

LICENSE.txt  TEST  test_labels.csv  TRAIN  train_labels.csv


In [15]:
data_dir = Path('./TRAIN')
data_dir

Path('TRAIN')

In [16]:
data_dir.ls()
# with xml and jpg files in one folder

(#4720) [Path('TRAIN/ed00fa5f-60af-4767-b8e8-f5405fd6eb44.xml'),Path('TRAIN/soybean-plant-seedling.jpg'),Path('TRAIN/fi000294.jpg'),Path('TRAIN/gcp_A-2_Late_Blight_of_potatoes_callout1.xml'),Path('TRAIN/CRABAPPLE2_leaves.jpg'),Path('TRAIN/img_8042.xml'),Path('TRAIN/Blackberry-black-raspberry3-1024x665.xml'),Path('TRAIN/tomato-leaf-6291945.jpg'),Path('TRAIN/southern-rust-close.xml'),Path('TRAIN/stock-photo-leaf-blueberry-57690040.jpg')...]

In [17]:
template_record = ObjectDetectionRecord()

In [22]:
class PlantParser(Parser):
  def __init__(self, template_record, data_dir):
    super().__init__(template_record = template_record)

    self.data_dir = data_dir
    self.df = train_labels # train_labels = pd.read_csv('train_labels.csv')
    self.class_map = ClassMap(list(self.df['label'].unique()))

  def __iter__(self) -> Any:
    for o in self.df.itertuples():
      yield o

  def __len__(self) -> int:
    return len(self.df)

  def record_id(self, o) -> Hashable:
    return o.filename

  def parse_fields(self, o, record, is_new):
    if is_new:
      record.set_filepath(self.data_dir / 'images'/ o.filename)
      record.set_img_size(ImgSize(width = o.width, height = o.height))
      record.detection.set_class_map(self.class_map)

    record.detection.add_bboxes([BBox.from_xyxy(o.xmin, o.ymin, o.xmax, o.ymax)])
    record.detection.add_labels([o.label])

#  records could not be autofixed, file does not exist

In [23]:
parser = PlantParser(template_record, data_dir)

train_records, valid_records = parser.parse()

  0%|          | 0/8469 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:136


  0%|          | 0/1876 [00:00<?, ?it/s]

WARNING  - (record_id: 220) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/grayleafspotinbred.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 2249) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/septoria-tomato-leaf.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 802) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/P6120019.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 777) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/05.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 2262) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/011.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 1332) - 🚫 Record could not be autofixed and will 

  0%|          | 0/469 [00:00<?, ?it/s]

WARNING  - (record_id: 567) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/apple-tree-branch-blossom-plant-fruit-berry-leaf-flower-food-green-produce-evergreen-flora-sad-shrub-apples-branch-with-apples-flowering-plant-rose-family-acerola-malpighia-woody-plant-land-plant-928225.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 2120) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/CTG514794.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 1822) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/tomato_leaf-mold_04_zoom.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 1503) - 🚫 Record could not be autofixed and will be removed because: File 'TRAIN/images/IMG_7182_original.JPG?1493665266.jpg' does not exist | icevision.core.record:autofix_records:104
WARNING  - (record_id: 1424) -

In [ ]:
# try using annot within the folder

In [30]:
class PlantParser2(Parser):
  def __init__(self, template_record, data_dir):
    super().__init__(template_record = template_record)

    self.data_dir = data_dir
    #self.df = train_labels # train_labels = pd.read_csv('train_labels.csv')
    self.df = data_dir ## patterning after voc format
    #self.class_map = ClassMap(list(self.df['label'].unique()))
    self.class_map = ClassMap(list(train_labels.label.unique()))# based on csv

  def __iter__(self) -> Any:
    for o in self.df.itertuples():
      yield o

  def __len__(self) -> int:
    return len(self.df)

  def record_id(self, o) -> Hashable:
    return o.filename

  def parse_fields(self, o, record, is_new):
    if is_new:
      record.set_filepath(self.data_dir / 'images'/ o.filename)
      record.set_img_size(ImgSize(width = o.width, height = o.height))
      record.detection.set_class_map(self.class_map)

    record.detection.add_bboxes([BBox.from_xyxy(o.xmin, o.ymin, o.xmax, o.ymax)])
    record.detection.add_labels([o.label])

#  1.  records could not be autofixed, file does not exist
#  2.  self.df changed to data_dir: 
#  ----> 8     self.class_map = ClassMap(list(self.df['label'].unique()))
#         TypeError: 'PosixPath' object is not subscriptable (for classmap?)


In [31]:
# with df set to data_dir
parser = PlantParser2(template_record, data_dir)

train_records, valid_records = parser.parse()

0it [00:00, ?it/s]

AttributeError: ignored

In [ ]:
### try to get tensorflow format

In [24]:
train_records[0]

IndexError: ignored

In [ ]:
show_record(train_records[0], class_map=class_map, font_size=25,
            label_color = '#ffff00')

In [ ]:
show_record(train_records[1], class_map=class_map, font_size=25,
            label_color = '#ffff00',
            figsize=(14,10))

In [ ]:
show_records(train_records[:3], ncols =3, class_map=class_map)